The goal of this notebook is to test the GP with the Gaussian conditional likelihood assumption.

In [1]:
import importlib
import itertools
import numpy as np
import matplotlib.pyplot as plt
import torch

%matplotlib inline

In [2]:
import razor_data
import plotting
import gp
importlib.reload(razor_data);
importlib.reload(plotting);

In [3]:
binned_data = razor_data.get_binned_data_1d(num_mr_bins=100)

In [4]:
# Squared exponential kernel
ell = 100
alpha = 1
kernel = gp.SquaredExponentialKernel(ell, alpha)

In [5]:
# Try it for data in one box
test_data = binned_data['MultiJet'][1]
U = test_data['u']
Y = test_data['y']

In [6]:
G = gp.GaussianLikelihoodGP(kernel, U, Y)

In [7]:
G.predict(U)

(Variable containing:
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 [torch.FloatTensor of size 100], Variable containing:
   nan   nan   nan  ...    nan   nan   nan
   nan   nan   nan  ...    nan   nan   nan
   nan   nan   nan  ...    nan   nan   nan
        ...          ⋱          ...       
   nan   nan   nan  ...    nan   nan   nan
   nan   nan   nan  ...    nan   nan   nan
   nan   nan   nan  ...    nan   nan   nan
 [torch.FloatTensor of size 100x100])